In [1]:
import pandas as pd

In [21]:
jinv_minimal_polynomials = pd.read_pickle('../data/dataframes/hilb_lifts.pk')
eccountdata = pd.read_pickle('../data/dataframes/elliptic_curve_counts.pk')

In [22]:
eccountdata.loc[eccountdata.degree==1]

,TraceFrob,Prime,Discriminant,DiscriminantFac,jInvariants,degree
ap,,,,,,
"(1, 5)",1,5,-19,"(-19, 1)",[4],1
"(3, 5)",3,5,-11,"(-11, 1)",[2],1
"(4, 5)",4,5,-4,"(-4, 1)",[3],1
"(3, 7)",3,7,-19,"(-19, 1)",[1],1
"(5, 7)",5,7,-3,"(-3, 1)",[0],1
...,...,...,...,...,...,...
"(251, 15791)",251,15791,-163,"(-163, 1)",[10766],1
"(252, 15877)",252,15877,-4,"(-4, 1)",[1728],1
"(253, 16007)",253,16007,-19,"(-19, 1)",[11656],1


In [14]:
hilblifts

,coefs,ReducedDisc,Conductor,deg
d,,,,
-3,"[0, 1]",-3,1,1
-4,"[-1728, 1]",-4,1,1
-8,"[-8000, 1]",-8,1,1
-11,"[32768, 1]",-11,1,1
-12,"[0, -54000, 1]",-3,2,2
...,...,...,...,...
-30067,[-93541070534412252588715194722936575722175679...,-30067,1,14
-32323,[638401687605604061390314876094254404368703460...,-32323,1,15
-34483,[148403539327448096253508622631783924020320678...,-34483,1,15


In [19]:
eccountdata['degree'] = eccountdata.degree *(1+(eccountdata.TraceFrob==0))

In [20]:
eccountdata[['TraceFrob','Prime', 'Discriminant', 'DiscriminantFac', 'jInvariants',
       'degree']].to_pickle('../data/dataframes/elliptic_curve_counts.pk')